# Have a conversation with your PLG data

In [ ]:
!pip install openai --quiet
!pip install langchain --quiet

In [ ]:
import os
import getpass
import openai
from sqlalchemy import *

db_conn = create_engine(connection_url)

os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

## Generate Embeddings Function

In [ ]:
openai.api_key = os.getenv('OPENAI_API_KEY')
model_id = 'text-embedding-ada-002'

def create_embedding(content,model_id):
    try:
        response = openai.Embedding.create(input=str(content),model=model_id)
        embedding = response['data'][0]['embedding']
        tokens = response['usage']['total_tokens']
        status = 'success'
        return embedding
    except Exception as e:
        print(e)
        embedding = ''
        tokens = 0
        status = 'failed'
        return embedding

## Chat w/ your data

In [ ]:
query = input("How can I help you?")

query_embedding = create_embedding(query,model_id)

sql_stmt = """
    SELECT
        customers.user_id,
        customers.first_name,
        customers.last_name,
        customers.email,
        actions.event,
        actions.seg_timestamp,
        DOT_PRODUCT(JSON_ARRAY_PACK(%s), actions.embedding) AS similarity
    FROM actions
    INNER JOIN customers ON customers.user_id = actions.user_id
    ORDER BY similarity DESC
    LIMIT 150;
"""

results = db_conn.execute(sql_stmt, str(query_embedding))

r_list = []
for r in results:
    r_dict = {}
    r_dict['user_id'] = r['user_id']
    r_dict['first_name'] = r['first_name']
    r_dict['last_name'] = r['last_name']
    r_dict['email'] = r['email']
    r_dict['event'] = r['event']
    r_dict['seg_timestamp'] = r['seg_timestamp']
    r_dict['similarity'] = r['similarity']
    r_list.append(r_dict)

data = str(r_list)
data = data[:9000]

prompt = f"The user asked: {query}. From the following context, please help answer the query: {data}"

response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": "You are a helpful assistant working for Super Duper ToDo who can help us sort out details about our customers, write emails about our award winning todo app, and who is excellent at understanding how to get users to re-engage with our application. You will always deliver your answers in plain language without referencing the raw data output. When refering to users, always use their first and last name, not their user_id."},
        {"role": "user", "content": prompt}
    ]
)

print(response['choices'][0]['message']['content'])
